<a href="https://colab.research.google.com/github/Sabrina-Hendricks/DS4002-Group13-Project2/blob/main/Scripts/Project2MI3_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

Import data from Github

In [5]:
url = 'https://raw.githubusercontent.com/Sabrina-Hendricks/DS4002-Group13-Project2/refs/heads/main/Data/HealthCareOriginalData.csv'
df = pd.read_csv(url)
df.head()

,Unnamed: 0,2004,2004.1,2004.2,2004.3,2005,2005.1,2005.2,2005.3,2006,...,2022,2022.1,2022.2,2022.3,2023,2023.1,2023.2,2023.3,2024,2024.1
0,NaN,Q1,Q2,Q3,Q4,Q1,Q2,Q3,Q4,Q1,...,Q1,Q2,Q3,Q4,Q1,Q2,Q3,Q4,Q1,Q2
1,Health care,1202.0,1226.0,1252.5,1275.0,1291.9,1309.5,1333.2,1347.6,1362.2,...,2753.8,2772.7,2835.3,2901.0,2983.3,3029.9,3068.6,3148.8,3233.6,3274.3
2,Outpatient services,575.6,585.0,594.6,601.8,613.0,621.1,628.1,635.4,639.2,...,1285.3,1306.6,1321.2,1346.6,1402.7,1420.2,1444.2,1481.1,1519.8,1554.7
3,Physician services (44),306.6,311.7,316.7,321.9,327.6,332.3,336.3,339.5,341.8,...,662.4,672.2,681.4,692.3,717.7,733.5,736.8,751.4,765.7,785.6
4,Dental services (45),81.0,82.5,84.2,85.4,86.7,87.7,88.1,89.0,89.7,...,158.6,160.4,163.4,165.7,170.7,171.3,173.3,175.4,180.2,183.9


Clean the data to make it in a readable/analyzable format

In [6]:
# Filter rows to keep only the specified services
services_to_keep = ['Physician services (44)', 'Dental services (45)', 'Paramedical services (46)',
                    'Hospitals (51)', 'Nursing homes (52)']
df_filtered = df[df['Unnamed: 0'].isin(services_to_keep)]

# Melt the dataframe to get it into long format for analysis
df_melted = df_filtered.melt(id_vars=['Unnamed: 0'], var_name='year_quarter', value_name='spending')

# Split 'year_quarter' into separate year and quarter columns
df_melted['year'] = df_melted['year_quarter'].apply(lambda x: x.split('.')[0])
df_melted['quarter'] = df_melted['year_quarter'].apply(lambda x: 'Q' + str(int(x.split('.')[1])+1) if '.' in x else 'Q1')

# Ensure that the 'spending' column is numeric
df_melted['spending'] = pd.to_numeric(df_melted['spending'], errors='coerce')

# Drop unnecessary 'year_quarter' column
df_melted = df_melted.drop(columns=['year_quarter'])

# Clean up the service names (remove the numbers in parentheses)
df_melted['Unnamed: 0'] = df_melted['Unnamed: 0'].apply(lambda x: re.sub(r'\s\(\d+\)', '', x))

# Map services to categories
category_map = {
    'Physician services': 'Outpatient services',
    'Dental services': 'Outpatient services',
    'Paramedical services': 'Outpatient services',
    'Hospitals': 'Hospital and nursing home services',
    'Nursing homes': 'Hospital and nursing home services'
}

# Add a 'category' column based on the service
df_melted['category'] = df_melted['Unnamed: 0'].map(category_map)

# Rename 'Unnamed: 0' to 'service' for clarity
df_melted = df_melted.rename(columns={'Unnamed: 0': 'service'})

# Rearrange columns to desired order
df_melted = df_melted[['category', 'service', 'year', 'quarter', 'spending']]

df_melted #this is the clean dataset - use this code for datacleaning***

,category,service,year,quarter,spending
0,Outpatient services,Physician services,2004,Q1,306.6
1,Outpatient services,Dental services,2004,Q1,81.0
2,Outpatient services,Paramedical services,2004,Q1,188.0
3,Hospital and nursing home services,Hospitals,2004,Q1,518.4
4,Hospital and nursing home services,Nursing homes,2004,Q1,108.0
...,...,...,...,...,...
405,Outpatient services,Physician services,2024,Q2,785.6
406,Outpatient services,Dental services,2024,Q2,183.9
407,Outpatient services,Paramedical services,2024,Q2,585.1
408,Hospital and nursing home services,Hospitals,2024,Q2,1475.0


Export the dataset to a csv and upload to github manually after

You should not need to save the csv and upload again - it is in Github and all analysis scripts reference it.

In [7]:
df_melted.to_csv('Cleaned_Data.csv', index=False)